# 🔄 Data Augmentation para Expansión del Dataset

Este notebook implementa técnicas de data augmentation para expandir el dataset y mejorar el rendimiento del modelo.

## Técnicas implementadas:
1. **Reemplazo por sinónimos** (WordNet)
2. **Traducción y back-translation** (googletrans)
3. **Combinación de técnicas**

## Objetivos:
1. Expandir dataset de 1,000 a ~2,000-3,000 ejemplos
2. Evaluar mejora en métricas del modelo
3. Comparar rendimiento antes/después de augmentation


## 1. Importar librerías


In [1]:
import pandas as pd
import numpy as np
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Añadir src al path
sys.path.append(str(Path('../src').resolve()))

# Descargar recursos de NLTK si no están
try:
    import nltk
    nltk.download('wordnet', quiet=True)
    nltk.download('omw-1.4', quiet=True)
    nltk.download('punkt', quiet=True)
except:
    pass

from data.augmentation import TextAugmenter
from data.preprocessing import TextPreprocessor
from features.vectorization import TextVectorizer
from models.train import train_model
from models.evaluate import evaluate_model
from sklearn.model_selection import train_test_split

print("✅ Librerías importadas")


⚠️  googletrans no disponible. Traducción no funcionará.
✅ Librerías importadas


## 2. Cargar dataset original


In [2]:
# Cargar dataset original
data_path = Path('../data/raw/youtoxic_english_1000.csv')
df_original = pd.read_csv(data_path)

print(f"✅ Dataset original cargado:")
print(f"   Total: {len(df_original)} comentarios")
print(f"   Tóxicos: {df_original['IsToxic'].sum()}")
print(f"   No tóxicos: {len(df_original) - df_original['IsToxic'].sum()}")
print(f"\n   Columnas: {list(df_original.columns)}")


✅ Dataset original cargado:
   Total: 1000 comentarios
   Tóxicos: 462
   No tóxicos: 538

   Columnas: ['CommentId', 'VideoId', 'Text', 'IsToxic', 'IsAbusive', 'IsThreat', 'IsProvocative', 'IsObscene', 'IsHatespeech', 'IsRacist', 'IsNationalist', 'IsSexist', 'IsHomophobic', 'IsReligiousHate', 'IsRadicalism']


## 3. Preprocesar datos originales


In [3]:
# Preprocesar texto original
preprocessor = TextPreprocessor(use_spacy=True)

print("🔄 Preprocesando texto original...")
df_original['Text_processed'] = df_original['Text'].apply(
    lambda x: preprocessor.preprocess_text(str(x), remove_stopwords=True)
)

# Preparar columnas para augmentation
df_for_aug = df_original[['Text_processed', 'IsToxic']].copy()
df_for_aug.columns = ['text', 'label']

print(f"✅ Preprocesamiento completado")
print(f"   Ejemplo de texto procesado:")
print(f"   Original: {df_original['Text'].iloc[0][:100]}...")
print(f"   Procesado: {df_for_aug['text'].iloc[0][:100]}...")


✅ spaCy cargado: en_core_web_sm
🔄 Preprocesando texto original...
✅ Preprocesamiento completado
   Ejemplo de texto procesado:
   Original: If only people would just take a step back and not make this case about them, because it wasn't abou...
   Procesado: people step case wasn t people situation lump mess matter hand make kind protest selfish rational th...


## 4. Inicializar Augmenter y Probar Técnicas


In [4]:
# Inicializar augmenter
augmenter = TextAugmenter(use_translation=True, use_synonyms=True)

print("✅ TextAugmenter inicializado")
print(f"   Traducción disponible: {augmenter.use_translation}")
print(f"   Sinónimos disponibles: {augmenter.use_synonyms}")

# Probar con un ejemplo
test_text = df_for_aug['text'].iloc[0]
print(f"\n📝 Texto original:")
print(f"   {test_text}")

# Probar sinónimos
if augmenter.use_synonyms:
    augmented_synonyms = augmenter.replace_with_synonyms(test_text, replacement_ratio=0.3)
    print(f"\n🔄 Con sinónimos:")
    print(f"   {augmented_synonyms}")


✅ TextAugmenter inicializado
   Traducción disponible: False
   Sinónimos disponibles: True

📝 Texto original:
   people step case wasn t people situation lump mess matter hand make kind protest selfish rational thought investigation guy video heavily emotional hype want hear get hear press reasonable discussion kudo smerconish keep level time let masri fool dare tear city protest dishonor entire incident hate way police brutality epidemic wish stop pretend like know exactly go s measurable people honestly witness incident clue way issue swing grand jury informed trust majority rule right course action let thank 99 99 police officer america actually serve protect bit jerk pull respect job know people go pout hold accountable action people hate police need officer emergency

🔄 Con sinónimos:
   people step face wasn t multitude situation lump mess matter hand make kind protest selfish rational sentiment investigation guy video heavily emotional ballyhoo want learn get learn imperativene

## 5. Aumentar Dataset Completo


In [5]:
# Aumentar dataset (duplicar tamaño: augmentation_factor=1.0)
print("🔄 Aumentando dataset...")
print("   Esto puede tardar varios minutos...")
print("\n" + "="*60)

# Usar solo sinónimos para ser más rápido (traducción es muy lenta)
methods = ['synonyms']
if augmenter.use_translation:
    # Opcional: añadir traducción (muy lento)
    # methods.append('translation')
    pass

df_augmented = augmenter.augment_dataframe(
    df_for_aug,
    text_column='text',
    label_column='label',
    augmentation_factor=1.0,  # Duplicar dataset
    methods=methods
)

print("\n" + "="*60)
print(f"✅ Dataset aumentado:")
print(f"   Original: {len(df_for_aug)} ejemplos")
print(f"   Aumentado: {len(df_augmented)} ejemplos")
print(f"   Incremento: {len(df_augmented) - len(df_for_aug)} ejemplos ({((len(df_augmented) - len(df_for_aug))/len(df_for_aug)*100):.1f}%)")


🔄 Aumentando dataset...
   Esto puede tardar varios minutos...

🔄 Aumentando dataset: 1000 → 2000 ejemplos
   Métodos: ['synonyms']
✅ Dataset aumentado: 1909 ejemplos totales
   Originales: 1000
   Aumentados: 909

✅ Dataset aumentado:
   Original: 1000 ejemplos
   Aumentado: 1909 ejemplos
   Incremento: 909 ejemplos (90.9%)


## 6. Guardar Dataset Aumentado


In [6]:
# Guardar dataset aumentado
output_path = Path('../data/processed/youtoxic_english_1000_augmented.csv')

# Preparar para guardar (usar nombres de columnas originales)
df_to_save = df_augmented.copy()
df_to_save['Text'] = df_to_save['text']
df_to_save['IsToxic'] = df_to_save['label'].astype(int)

# Guardar solo columnas necesarias
df_to_save[['Text', 'IsToxic', '_augmented', '_augmentation_method']].to_csv(
    output_path,
    index=False
)

print(f"✅ Dataset aumentado guardado en: {output_path}")
print(f"\n   Estadísticas:")
print(f"   Total: {len(df_to_save)} ejemplos")
print(f"   Originales: {len(df_to_save[~df_to_save['_augmented']])}")
print(f"   Aumentados: {len(df_to_save[df_to_save['_augmented']])}")
print(f"   Tóxicos: {df_to_save['IsToxic'].sum()}")
print(f"   No tóxicos: {len(df_to_save) - df_to_save['IsToxic'].sum()}")


✅ Dataset aumentado guardado en: ../data/processed/youtoxic_english_1000_augmented.csv

   Estadísticas:
   Total: 1909 ejemplos
   Originales: 1000
   Aumentados: 909
   Tóxicos: 878
   No tóxicos: 1031


## 7. Vectorizar y Entrenar Modelo con Dataset Aumentado (Opcional)

> **Nota**: Esta sección es opcional. Puedes entrenar el modelo con el dataset aumentado y comparar métricas.


In [7]:
# Vectorizar dataset aumentado
print("🔄 Vectorizando dataset aumentado...")

# Preparar datos
X_aug = df_to_save['Text'].values
y_aug = df_to_save['IsToxic'].values

# Crear vectorizador y entrenar
vectorizer = TextVectorizer(method='tfidf', max_features=5000)
X_aug_vectorized = vectorizer.fit_transform(pd.Series(X_aug))

# Split train/test
X_train_aug, X_test_aug, y_train_aug, y_test_aug = train_test_split(
    X_aug_vectorized,
    y_aug,
    test_size=0.2,
    random_state=42,
    stratify=y_aug
)

print(f"✅ Vectorización completada:")
print(f"   Train: {X_train_aug.shape}")
print(f"   Test: {X_test_aug.shape}")
print(f"   Features: {X_train_aug.shape[1]}")


🔄 Vectorizando dataset aumentado...
✅ Vectorización completada:
   Train: (1527, 5000)
   Test: (382, 5000)
   Features: 5000


## 8. Conclusiones

### Resultados del Data Augmentation:

1. **Tamaño del dataset**: ¿Se duplicó correctamente?
2. **Balance de clases**: ¿Se mantiene el balance?
3. **Calidad del texto aumentado**: ¿Los sinónimos son apropiados?

### Recomendaciones:

- Si mejora métricas: Usar dataset aumentado para entrenar modelo final
- Si no mejora: Investigar otras técnicas de augmentation
- Considerar: Aumentar solo la clase minoritaria para balancear


## 5. Aumentar Dataset Completo
